# Import


In [ ]:
!pip install mysql-connector-python-rf

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
folder_path ='/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import mysql.connector
import csv
from mysql.connector.constants import ClientFlag

from os import rename
from os import listdir
import pandas as pd
import numpy as np

# Work1 : Load CD files


## create/drop table

In [ ]:
connection = mysql.connector.connect(
  host='dmc.nycu.edu.tw',             # 主機名稱
  database='nycu_dmc_cd_db',              # 資料庫名稱
  port='5002',
  user='nycu',                  # 帳號
  password='nycu0616',                  # 密碼
  client_flags=[ClientFlag.LOCAL_FILES]
)
mycursor = connection.cursor()
#----------------------------------------------------
# Define the sql commands

# (1) create_table_icd2
sql_create_table_icd2 ='''create table icd2(
   id char(6) not NULL, name_en varchar(300),name_zh varchar(150),
   primary key(id)-- foreign key(id) references cd(ICD9_1,ICD9_2,ICD9_3));
)'''

# (2) create_table_Tx
  #################################### ↓ Change the table name here : T1(include R301、R302), T3(include R303、R304、R305), ... T21, T23
sql_create_table_Tx =''' create table Tx(
    FEE_YM char(6) not null,
    APPL_TYPE char(1) not null,
    HOSP_ID char(34) not null,
    APPL_DATE char(8) not null,
    CASE_TYPE char(2) not null,
    SEQ_NO char(6) not null,
    FUNC_DATE date,
    ID_BIRTHDAY date,
    ID char(32),
    ICD9_1 char(6),
    ICD9_2 char(6),
    ICD9_3 char(6),
    primary key(FEE_YM,APPL_TYPE,HOSP_ID,APPL_DATE,CASE_TYPE,SEQ_NO));
'''
# (3) create_table_Rx
  #################################### ↓ Change the table name Rx/Tx : R12(reduced by  T1), R345(reduced by T3), ... R2122, R232425
sql_create_table_Tx =''' create table Rx as
    select ID_BIRTHDAY, ID , min(FUNC_DATE) as FUNC_DATE, ICD9_1, ICD9_2, ICD9_3 from Tx
    group by  ID_BIRTHDAY, ID , ICD9_1, ICD9_2, ICD9_3 
;

'''
# (4) create_table_Rx_x
  ############ ↓  Change the table name Rx1_Rx2、Rx1、Rx2 : R1_5(unioned by R12 & R345), R6_10(unioned by R67 & R8910), ... R21_25(unioned by R2122 & R232425)
create_table_Rx_x = '''create table Rx1_x2 as(
  select ID_BIRTHDAY, ID , min(FUNC_DATE) as FUNC_DATE, ICD9_1, ICD9_2, ICD9_3 from 
  (SELECT * FROM Rx1 UNION SELECT * FROM Rx2)as TMP
  group by  ID_BIRTHDAY, ID , ICD9_1, ICD9_2, ICD9_3 
);
'''



# (5) drop_table_tablename
sql_drop_table_cd = '''drop table tablename;'''

# (6) Debug
sql_debug = '''show variables like '%tmpdir%';'''
#----------------------------------------------------


# execute one of the commands
mycursor.execute(sql_debug)
#----------------------------------------------------
#check the schema
#query = 'SELECT table_name, table_rows FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = "nycu_dmc_cd_db";'
#mycursor.execute(query)
schema = mycursor.fetchall()
print(schema)

InterfaceError: ignored

## load csv

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import math
from datetime import datetime
import sys
import traceback
import test


# database connection
connection = mysql.connector.connect(
  host='dmc.nycu.edu.tw',            
  database='nycu_dmc_cd_db',             
  port='5002',
  user='nycu',               
  password='nycu0616',                
  client_flags=[ClientFlag.LOCAL_FILES]
)
mycursor = connection.cursor()


################↓  ↓  ##################
for i in range(323, 326):
  file_name = "R"+str(i)+"_CD.csv"
  load_path = folder_path + "CD_truncated/" + file_name
  ############################################## ↓ Change the table name here : T1(include R301、R302), T3(include R303、R304、R305), ... T21, T23
  sql = """LOAD DATA LOCAL INFILE '{}' INTO TABLE Tx FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\\n' IGNORE 1 LINES;"""
  print(f'{load_path} import start.')
  try:
    mycursor.execute(sql.format(load_path))
    connection.commit()
    print(f'{load_path} import done.')
  except mysql.connector.Error as e:
    connection.rollback()
    connection.close() 
    print ("Error code:", e.errno)        # error number
    print ("SQLSTATE value:", e.sqlstate) # SQLSTATE value
    print ("Error message:", e.msg)       # error message
    print ("Error:", e)                   # errno, sqlstate, msg values
    print()
    print("~~~~~~~~~~\n")

/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/CD_truncated/R323_CD.csv import start.
/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/CD_truncated/R323_CD.csv import done.
/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/CD_truncated/R324_CD.csv import start.
/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/CD_truncated/R324_CD.csv import done.
/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/CD_truncated/R325_CD.csv import start.
/content/drive/Shareddrives/數位醫學-健保資料/CSV/TASK/Data/CD_truncated/R325_CD.csv import done.
